In [ ]:
%load_ext autoreload
%autoreload 2

from ageself.filter_faces_eyetracking_functions import smooth_running_median, build_eye_tracking_dataset
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import glob
import numpy as np

import plotly.graph_objs as go
from plotly.subplots import make_subplots
import tqdm

In [ ]:
# Base Face functions
# Function to extract a single frame by index from a video
def load_frame_at(path, frame_idx):
    """
    Load a specific frame from the video without preloading all frames.

    Args:
        path (str): Path to the video file.
        frame_idx (int): Zero-based index of the desired frame.

    Returns:
        frame (np.ndarray): BGR image array of the specified frame.
    """
    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video file {path}")

    # Seek to the frame index
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ret, frame = cap.read()
    cap.release()
    if not ret:
        raise IndexError(f"Frame {frame_idx} cannot be read")
    return frame

# Function to plot a frame with bounding boxes
def plot_frame_with_boxes(frame, annotations, figsize=(8, 6)):
    """
    Plot a single video frame with rectangle annotations.

    Args:
        frame (np.ndarray): BGR image array.
        annotations (pd.DataFrame): Subset of box_annotation_df for one frame.
        figsize (tuple): Figure size.
    """
    # Convert BGR to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    fig, ax = plt.subplots(1, figsize=figsize)
    ax.imshow(frame_rgb)
    ax.axis('off')

    # Draw each box
    for _, row in annotations.iterrows():
        x, y, w, h = row.x_l, row.y_l, row.width, row.height
        age = row.age_class
        gender = row.gender
        rect = patches.Rectangle(
            (x, y), w, h,
            linewidth=1,
            edgecolor='red',
            facecolor='none'
        )
        ax.add_patch(rect)

    plt.tight_layout()
    return fig, ax

def scale_boxes(annotations, scale_factor = 1):
    """
    Scale bounding boxes in-place by a given factor, keeping the same center.

    Args:
        annotations (pd.DataFrame): DataFrame with columns [x_l, y_l, width, height].
        scale_factor (float): Scale multiplier (e.g., 2.0 doubles box size).

    Returns:
        pd.DataFrame: New DataFrame with scaled x_l, y_l, width, height.
    """
    df = annotations.copy().reset_index(drop=True)
    # Calculate centers
    cx = df.x_l + df.width / 2.0
    cy = df.y_l + df.height / 2.0

    # Scale sizes
    new_w = df.width * scale_factor
    new_h = df.height * scale_factor

    # Compute new top-left
    df['x_l'] = cx - new_w / 2.0
    df['y_l'] = cy - new_h / 2.0
    df['width'] = new_w
    df['height'] = new_h

    return df


# Eye Tracking FUnctions


def plot_eye_tracking_data(data_eye_tracking):
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        row_heights=[0.5, 0.5],
        vertical_spacing=0.1,
        subplot_titles=("X Position over Time", "Y Position over Time")
    )

    # X pos trace
    fig.add_trace(
        go.Scatter(
            x=data_eye_tracking["timestamp"],
            y=data_eye_tracking["pos_x"],
            mode="lines+markers",
            name="pos_x"
        ),
        row=1, col=1
    )

    # Y pos trace
    fig.add_trace(
        go.Scatter(
            x=data_eye_tracking["timestamp"],
            y=data_eye_tracking["pos_y"],
            mode="lines+markers",
            name="pos_y"
        ),
        row=2, col=1
    )

    # Layout tweaks
    fig.update_layout(
        height=600, width=800,
        title="Interactive Eye‑Tracking → X and Y Positions",
        xaxis_title="Timestamp",
        yaxis=dict(title="Position"),
        hovermode="x unified"
    )

    
    # fig.update_yaxes(range=[-2000, 2000], row=1, col=1)
    # fig.update_yaxes(range=[-2000, 2000], row=2, col=1)

    # Show it!
    fig.show()


def make_lag_diff(data_eye_tracking: pd.DataFrame, lag=1) -> pd.DataFrame:
    """
    Compute lag-n differences for X and Y eye‑tracking positions.

    Args:
        data_eye_tracking (pd.DataFrame):  
            Must contain columns ['world_index', 'pos_x', 'pos_y'].
        lag (int):  
            Number of steps back to subtract (e.g. lag=1 gives frame-to-frame diffs).

    Returns:
        pd.DataFrame: with columns ['world_index', 'delta_x', 'delta_y'], length = len(data_eye_tracking) - lag
    """
    df = data_eye_tracking.copy().reset_index(drop=True)

    # Compute shifted series
    df['pos_x_prev'] = df['pos_x'].shift(lag)
    df['pos_y_prev'] = df['pos_y'].shift(lag)

    # Compute deltas
    df['pos_x'] = df['pos_x'] - df['pos_x_prev']
    df['pos_y'] = df['pos_y'] - df['pos_y_prev']

    # Align world_index to the current frame (the later one)
    df_out = df.loc[lag:, ['world_index', 'pos_x', 'pos_y']].reset_index(drop=True)
    return df_out


# Base Loading of required data

In [ ]:
base_path = "/usr/users/vhassle"

# Load seqs meta data
information_raw_df = pd.read_csv(os.path.join(base_path,"datasets/Wortschatzinsel/head_mounted_data/scene_view_creation_df.csv"))
decision_df = pd.read_csv(os.path.join(base_path,"datasets/Wortschatzinsel/head_mounted_data/final_IDs.csv"))
# Create one dataset that has the annotations from the facedetection boxes and their according labels 
information_filtered_df = pd.merge(information_raw_df, decision_df, left_on="scene_view_nr", right_on="ID", how="inner")


In [ ]:
base_eye_tracking_raw_path = os.path.join(base_path, "datasets/Wortschatzinsel/head_mounted_data/eye_tracking_annotations/valid")
eye_tracking_raw_paths = glob.glob(os.path.join(base_eye_tracking_raw_path, "*", "gaze_positions*.csv"))
eye_tracking_raw_paths.sort()
seq_names = [eye_tracking_raw_path.split("/")[-2] for eye_tracking_raw_path in eye_tracking_raw_paths]

# idx 0 is definetly correct!

idx = 0 # 15, 91, 2
seq_name = seq_names[idx]

video_path = os.path.join(base_path,"datasets/Wortschatzinsel/head_mounted_data/videos/valid", seq_name + ".mp4")
box_annotation_path = os.path.join(base_path,"model_outputs/Wortschatzinsel/age_gender_classification_reversed_from_results", seq_name + ".txt")

eye_tracking_raw_path = eye_tracking_raw_paths[idx]

# Load detection annotations (boxes)
box_annotation_df = pd.read_csv(
    box_annotation_path,
    header=None,
    names=["frame", "face_nuber_on_frame", "x_l", "y_l", "width", "height", "n1","n2","n3","n4","age_class", "gender"])

box_annotation_df.frame = box_annotation_df.frame - 1


# Load eyetracking data
# check if pupilcore or neon
is_neon = information_filtered_df[information_filtered_df.new_name == seq_name].neon.values[0]
data_eye_tracking, gaze_df = build_eye_tracking_dataset(seq_name, video_path, eye_tracking_raw_path, base_eye_tracking_raw_path, is_neon, window_ms= 100)



print(f"Number of eyetracking annotations: {len(data_eye_tracking)}")
print(f"seq_name : {seq_name}")



# Look on video to see face size

In [ ]:
# Raw eye‑tracking data
data_eye_tracking_subset = gaze_df#.iloc[1000:2000].copy()
print(data_eye_tracking_subset.shape)
plot_eye_tracking_data(data_eye_tracking_subset)


# Postprocessing

- First group by frame of the video to get at least a constant frame rate (that is what is used so far)

In [ ]:
base_eye_tracking_raw_path = os.path.join(base_path, "datasets/Wortschatzinsel/head_mounted_data/eye_tracking_annotations/valid")
eye_tracking_raw_paths = glob.glob(os.path.join(base_eye_tracking_raw_path, "*", "gaze_positions*.csv"))
eye_tracking_raw_paths.sort()
seq_names = [eye_tracking_raw_path.split("/")[-2] for eye_tracking_raw_path in eye_tracking_raw_paths]

# idx 0 is definetly correct!

idx = 1 # 15, 91, 2
seq_name = seq_names[idx]

video_path = os.path.join(base_path,"datasets/Wortschatzinsel/head_mounted_data/videos/valid", seq_name + ".mp4")
box_annotation_path = os.path.join(base_path,"model_outputs/Wortschatzinsel/age_gender_classification_reversed_from_results", seq_name + ".txt")

eye_tracking_raw_path = eye_tracking_raw_paths[idx]

# Load detection annotations (boxes)
box_annotation_df = pd.read_csv(
    box_annotation_path,
    header=None,
    names=["frame", "face_nuber_on_frame", "x_l", "y_l", "width", "height", "n1","n2","n3","n4","age_class", "gender"])

box_annotation_df.frame = box_annotation_df.frame - 1


# Load eyetracking data
# check if pupilcore or neon
is_neon = information_filtered_df[information_filtered_df.new_name == seq_name].neon.values[0]
data_eye_tracking, gaze_df = build_eye_tracking_dataset(seq_name, video_path, eye_tracking_raw_path, base_eye_tracking_raw_path, is_neon, window_ms= 100)



print(f"Number of eyetracking annotations: {len(data_eye_tracking)}")
print(f"seq_name : {seq_name}")


data_eye_tracking_subset = data_eye_tracking.copy()#[1000:2000]
data_eye_tracking_subset.timestamp = data_eye_tracking_subset.world_index
plot_eye_tracking_data(data_eye_tracking_subset)

In [ ]:
# Calculate lagged differences data plot

data_eye_tracking_subset =  data_eye_tracking.copy()[1000:2000]
print(data_eye_tracking_subset.shape)
diff_data = make_lag_diff(data_eye_tracking_subset)
diff_data["timestamp"] = diff_data["world_index"]
plot_eye_tracking_data(diff_data)


# Quantitive summary

### Movement of foucs (How far looks the person per Frame)

In [ ]:
base_eye_tracking_raw_path = os.path.join(base_path, "datasets/Wortschatzinsel/head_mounted_data/eye_tracking_annotations/valid")
eye_tracking_raw_paths = glob.glob(os.path.join(base_eye_tracking_raw_path, "*", "gaze_positions*.csv"))
eye_tracking_raw_paths.sort()
seq_names = [eye_tracking_raw_path.split("/")[-2] for eye_tracking_raw_path in eye_tracking_raw_paths]
seq_names.sort()

box_annotation_paths = glob.glob(os.path.join(base_path,"model_outputs/Wortschatzinsel/age_gender_classification_reversed_from_results","*.txt"))
box_annotation_paths.sort()

moving_table = []
for eye_tracking_raw_path, seq_name, box_annotation_path in tqdm.tqdm(zip(eye_tracking_raw_paths, seq_names, box_annotation_paths)):
    assert seq_name in box_annotation_path and seq_name in eye_tracking_raw_path, f"Mismatch in seq_name: {seq_name} not found in paths."
    # load the box annotations
    box_annotation_df = pd.read_csv(
        box_annotation_path,
        header=None,
        names=["frame", "face_nuber_on_frame", "x_l", "y_l", "width", "height", "n1","n2","n3","n4","age_class", "gender"])

    # check if pupilcore or neon
    is_neon = information_filtered_df[information_filtered_df.new_name == seq_name].neon.values[0]
    data_per_frame_eye, gaze_df = build_eye_tracking_dataset(seq_name, video_path, eye_tracking_raw_path, base_eye_tracking_raw_path, is_neon, window_ms= 100)

    moving = []
    moving.append(seq_name)
    moving.append(is_neon)

    box_eye_annotation_df = pd.merge(box_annotation_df, data_per_frame_eye, how='outer', left_on='frame', right_on='world_index')
    box_eye_annotation_df['frame'] = box_eye_annotation_df['frame'].combine_first(box_eye_annotation_df['world_index'])

    #optional smooth it for the statistics
    # if not is_neon:
    #     box_eye_annotation_df = smooth_running_mean(box_eye_annotation_df, window_ms=300, hz=30)

    #calculate the difference between two timeframes:
    box_lagged = make_lag_diff(box_eye_annotation_df, lag=1)
    movement_x = box_lagged["pos_x"].abs().mean()
    movement_y = box_lagged["pos_y"].abs().mean()
    moving.append(movement_x)
    moving.append(movement_y)

    moving_table.append(moving)

    


In [ ]:
overview_moving = pd.DataFrame(moving_table, columns=["seq_name", "is_neon", "x_change", "y_change"])
means = overview_moving.drop(columns=["seq_name"]).groupby("is_neon").mean()

print(means)

fig, axes = plt.subplots(2, 1, figsize=(10, 8), tight_layout=True)
for neon_flag, df_group in overview_moving.groupby("is_neon"):
    label = f"is_neon={neon_flag}"
    # x_change histogram on top row
    axes[0].hist(df_group["x_change"], bins=30, alpha=0.6, label=label)
    # y_change histogram on bottom row
    axes[1].hist(df_group["y_change"], bins=30, alpha=0.6, label=label)

# customize axes
axes[0].set_title("Histogram of x_change by is_neon")
axes[1].set_title("Histogram of y_change by is_neon")
for ax in axes:
    ax.set_xlabel("Change value")
    ax.set_ylabel("Frequency")
    ax.legend()

plt.show()


## Validity measures applied on Videos:

In [ ]:
import os, glob, tqdm, numpy as np, pandas as pd

# ------------------------------------------------------------------
# gather files
# ------------------------------------------------------------------
base_eye_tracking_raw_path = os.path.join(
    base_path,
    "datasets/Wortschatzinsel/head_mounted_data/eye_tracking_annotations/valid"
)
eye_tracking_raw_paths = sorted(
    glob.glob(os.path.join(base_eye_tracking_raw_path, "*", "gaze_positions*.csv"))
)
seq_names = sorted([p.split("/")[-2] for p in eye_tracking_raw_paths])

box_annotation_paths = sorted(
    glob.glob(os.path.join(
        base_path,
        "model_outputs/Wortschatzinsel/age_gender_classification_reversed_from_results",
        "*.txt"))
)

# ------------------------------------------------------------------
# metrics per video
# ------------------------------------------------------------------
metrics_table = []           # each row: [seq_name, is_neon, n_empty_tail, pct_zero_change, pct_longest_zero]

for eye_path, seq_name, box_path in tqdm.tqdm(zip(eye_tracking_raw_paths,
                                                  seq_names,
                                                  box_annotation_paths)):
    assert seq_name in box_path and seq_name in eye_path, f"path mismatch: {seq_name}"

    # -------------------------
    # build gaze‑data per frame
    # -------------------------
    is_neon = information_filtered_df.loc[
        information_filtered_df.new_name == seq_name, "neon"
    ].values[0]

    data_eye_tracking, gaze_df = build_eye_tracking_dataset(
        seq_name, video_path, eye_path, base_eye_tracking_raw_path,
        is_neon, window_ms=0
    )

    # -------------------------------------------------------------
    # 1) valid mask: both coordinates present
    # -------------------------------------------------------------
    valid_mask = data_eye_tracking[["pos_x", "pos_y"]].notna().all(axis=1)
    valid_df   = data_eye_tracking.loc[valid_mask].copy()

    # -------------------------------------------------------------
    # 1. empty‑tail frames
    # -------------------------------------------------------------
    max_valid_idx   = valid_df["world_index"].max()
    max_world_index = data_eye_tracking["world_index"].max()
    n_empty_tail    = int(max_world_index - max_valid_idx)

    # -------------------------------------------------------------
    # 2. percentage of repeated gaze points
    # -------------------------------------------------------------
    zero_change = (
        (valid_df["pos_x"].diff().fillna(0) == 0) &
        (valid_df["pos_y"].diff().fillna(0) == 0)
    )
    pct_zero_change = 100 * zero_change.mean()

    # -------------------------------------------------------------
    # 3. longest continuous zero‑change stretch (percent of valid)
    # -------------------------------------------------------------
    groups           = zero_change.ne(zero_change.shift()).cumsum()
    longest_stretch  = zero_change.groupby(groups).sum().max() or 0
    pct_longest_zero = 100 * longest_stretch / len(valid_df)

    # -------------------------------------------------------------
    # collect row
    # -------------------------------------------------------------
    metrics_table.append([
        seq_name,
        bool(is_neon),
        n_empty_tail,
        pct_zero_change,
        pct_longest_zero
    ])

# ------------------------------------------------------------------
# final DataFrame
# ------------------------------------------------------------------
cols = ["seq_name", "is_neon", "empty_tail_frames", "pct_zero_change", "pct_longest_zero"]
metrics_df = pd.DataFrame(metrics_table, columns=cols)
print(metrics_df.head())


In [ ]:
pd.set_option("display.max_rows", 200)
metrics_df.head(91)


### Compare diferent bounding boxes

In [ ]:
import glob
import tqdm
base_path_bounding_boxes = "/usr/users/vhassle/model_outputs/Wortschatzinsel/detection_tracking_merged_v04/age_gender_classification_reversed_from_results"


result_table_ts = []
result_table_fn = []
for seq_name in tqdm.tqdm((seq_names)):
    paths = glob.glob(os.path.join(base_path_bounding_boxes, f"*{seq_name}*.csv"))
    paths.sort()

    result_row_ts = []
    result_row_fn = []
    result_row_ts.append(int(seq_name.split("_")[0]))
    result_row_fn.append(int(seq_name.split("_")[0]))

    for path in paths:
        result = pd.read_csv(path)
        # drop where timestamp is NaN
        result = result.dropna(subset=['timestamp'])
        per_frame_result = result.groupby('frame', as_index=False)['eye_in_box'].max()
        mean_cover = per_frame_result.eye_in_box.mean()
        result_row_ts.append(per_frame_result.eye_in_box.mean())

        result = result.dropna(subset=['face_nuber_on_frame'])
        per_frame_result = result.groupby('frame', as_index=False)['eye_in_box'].max()
        mean_cover = per_frame_result.eye_in_box.mean()
        result_row_fn.append(per_frame_result.eye_in_box.mean())   

        
    result_table_ts.append(result_row_ts)
    result_table_fn.append(result_row_fn)
        

result_table_df_ts = pd.DataFrame(result_table_ts, columns=["seq_name", "0", "5", "10","15", "20", "30", "40", "50", "60", "80", "100"])
result_table_df_fn = pd.DataFrame(result_table_fn, columns=["seq_name", "0", "5", "10","15", "20", "30", "40", "50", "60", "80", "100"])


    



In [ ]:
result

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# result_table_df_fn, result_table_df_ts
result_table_df = result_table_df_ts
result_table_df["is_neon"] = (result_table_df['seq_name'] >= 400) & (result_table_df['seq_name'] < 500)
result_table_df

import matplotlib.pyplot as plt

# 1) Compute means per is_neon (dropping the seq_name column)
means = result_table_df.drop(columns=['seq_name']).groupby('is_neon').mean()
print(means)


# ------------------------------------------------------------------
# 0)  Collect depth columns and convert to numeric order
# ------------------------------------------------------------------
coverage_cols = (
    result_table_df                # your DataFrame
      .columns
      .difference(['seq_name', 'is_neon'])   # keep only depth columns
      .tolist()
)
coverage_cols = sorted(coverage_cols, key=lambda x: int(x))   # numeric sort

x_vals = np.array([int(c) for c in coverage_cols])            # numeric x‑axis

# ------------------------------------------------------------------
# 1)  Compute required quantiles per class
# ------------------------------------------------------------------
quant_levels = [0, 0.25, 0.5, 0.75, 1.0]
quant = (
    result_table_df
      .groupby('is_neon')[coverage_cols]
      .quantile(quant_levels)        # MultiIndex (is_neon, quantile)
)

# ------------------------------------------------------------------
# 2)  Plot median line + shaded quantile bands
# ------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(10, 5))

for is_neon_flag, color in zip([False, True], ['tab:blue', 'tab:orange']):
    q0   = quant.loc[(is_neon_flag, 0.00)][coverage_cols]
    q25  = quant.loc[(is_neon_flag, 0.25)][coverage_cols]
    q50  = quant.loc[(is_neon_flag, 0.50)][coverage_cols]
    q75  = quant.loc[(is_neon_flag, 0.75)][coverage_cols]
    q100 = quant.loc[(is_neon_flag, 1.00)][coverage_cols]

    # outer (min‑max) band
    ax.fill_between(x_vals, q0, q100, color=color, alpha=0.10)
    # inner (IQR) band
    ax.fill_between(x_vals, q25, q75, color=color, alpha=0.25)
    # median line
    ax.plot(x_vals, q50, color=color, marker='o', label=f'Neon = {is_neon_flag}')

    #mean line
    mean_vals = result_table_df.loc[result_table_df['is_neon'] == is_neon_flag, coverage_cols].mean()
    ax.plot(x_vals, mean_vals, color=color, linestyle='--', label=f'Mean (Neon = {is_neon_flag})')

# ------------------------------------------------------------------
# 3)  Cosmetics
# ------------------------------------------------------------------
ax.set_xlabel('Sequencing depth category')
ax.set_ylabel('Coverage')
ax.set_title('Coverage distribution by depth (median + quantile bands)')
ax.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.6)
ax.legend(title='Group')
plt.tight_layout()
plt.show()